# Notebook for replicating the analyses in "Tennessee Trends: The State of Tennessee’s Economy"
### Richard W. Evans, September 2024
This notebook replicates the analyses by [Richard W. Evans](https://sites.google.com/site/rickecon) (@rickecon) in the September 2024 article on Tennessee business tax reform as part of the article "[Tennessee Trends: The State of Tennessee’s Economy](https://www.beacontn.org/tennessee-trends-the-state-of-tennessees-economy/)," by Ron Shultis, Jason Edmonds, and [Richard W. Evans](https://sites.google.com/site/rickecon), Beacon Center of Tennessee (Sep. 12, 2024).

A Google Colab version of this notebook that can be run from your browser in the cloud is available at [this link](https://colab.research.google.com/drive/1dDKVTEAm4drvNXgHpm617j-82X4xzxKh?usp=sharing).

## 1. Introduction
Before opening and running this notebook, make sure that you have downloaded or cloned the [`TN-BusTax2024`](https://github.com/OpenSourceEcon/TN-BusTax2024) repository (https://github.com/OpenSourceEcon/TN-BusTax2024) and created and activated the associated conda environment `tn-bustax2024-dev` in the `environment.yml` file. You can also run this notebook easily in the cloud by going to this [Google Colab version](https://colab.research.google.com/drive/1dDKVTEAm4drvNXgHpm617j-82X4xzxKh?usp=sharing) of this notebook.

In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import (
    ColumnDataSource, Title, Label, LabelSet, Legend, LegendItem,
    CategoricalColorMapper, ColorBar, HoverTool, NumeralTickFormatter,
    GeoJSONDataSource, FactorRange
)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.transform import factor_cmap
from bokeh.sampledata.us_states import data as states
from bokeh.transform import cumsum
from bokeh.palettes import Category20c

import ccc
import taxcalc as tc
import requests
import random
import json

import plotly.express as px
import plotly.io as pio
# set template for plotting
pio.templates.default = "plotly_white"

### 1.1. Figure 4. Rainy Day Fund and Total Reserves as Percentages of General-Fund Expenditures, Tennessee and 50-State Median: 2000-2023

Get the Pew state rainy day fund data. This does not include District of Columbia. The 2022 data is now final. The 2023 data is labeled as "Enacted" which means projected based on enacted legislation.

In [2]:
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data')
image_dir = os.path.join(cur_dir, 'images')

In [3]:
# Read in the Pew data on rainy day funds by year and state from worksheet
rain_totbal_path = (
    "https://github.com/OpenSourceEcon/TN-BusTax2024/blob/main/data/" +
    "ReservesBalancesData.xlsx?raw=true"
)
rain_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Rainy Day Fund Data",
    header=4,
    index_col=0,
    skipfooter=2
)

# Create a DataFrame of just the rainy day funds in $millions
# by state and by year
rain_dol_df = rain_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Estimated']
rain_dol_df = rain_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016': '2016',
    ' FY 2017': '2017',
    ' FY 2018': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022': '2022',
    'FY 2023 Estimated': '2023'
})
rain_dol_df

# Create a DataFrame of just the rainy day funds as a percent of
# general fund expenditures by state and by year
rain_pct_df = rain_df.loc[:, 'FY 2000.2':'FY 2023 Estimated.1']
rain_pct_df = rain_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.1': '2022',
    'FY 2023 Estimated.1': '2023'
})

In [4]:
# Read in the PEW data on total reserves and balances by year and state from
# worksheet
totbal_df = pd.read_excel(
    rain_totbal_path,
    sheet_name="Total Balances Data",
    header=5,
    index_col=0,
    skipfooter=2
)

# Create a DataFrame of just the total reserves and balances funds in $millions
# by state and by year
totbal_dol_df = totbal_df.loc[:'Wyoming', 'FY 2000.1':'FY 2023 Estimated.1']
totbal_dol_df = totbal_dol_df.rename(columns = {
    'FY 2000.1': '2000',
    'FY 2001.1': '2001',
    'FY 2002.1': '2002',
    'FY 2003.1': '2003',
    'FY 2004.1': '2004',
    'FY 2005.1': '2005',
    'FY 2006.1': '2006',
    'FY 2007.1': '2007',
    'FY 2008.1': '2008',
    'FY 2009.1': '2009',
    'FY 2010.1': '2010',
    'FY 2011.1': '2011',
    'FY 2012.1': '2012',
    'FY 2013.1': '2013',
    'FY 2014.1': '2014',
    'FY 2015.1': '2015',
    ' FY 2016.1': '2016',
    ' FY 2017.1': '2017',
    ' FY 2018.1': '2018',
    'FY 2019.1': '2019',
    'FY 2020.1': '2020',
    'FY 2021.1': '2021',
    'FY2022.1': '2022',
    'FY 2023 Estimated.1': '2023'
})

# Create a DataFrame of just the total reserves and balances funds as percent
# of general fund expenditures by state and by year
totbal_pct_df = totbal_df.loc[:, 'FY 2000.2':'FY 2023 Estimated.2']
totbal_pct_df = totbal_pct_df.rename(columns = {
    'FY 2000.2': '2000',
    'FY 2001.2': '2001',
    'FY 2002.2': '2002',
    'FY 2003.2': '2003',
    'FY 2004.2': '2004',
    'FY 2005.2': '2005',
    'FY 2006.2': '2006',
    'FY 2007.2': '2007',
    'FY 2008.2': '2008',
    'FY 2009.2': '2009',
    'FY 2010.2': '2010',
    'FY 2011.2': '2011',
    'FY 2012.2': '2012',
    'FY 2013.2': '2013',
    'FY 2014.2': '2014',
    'FY 2015.2': '2015',
    ' FY 2016.2': '2016',
    ' FY 2017.2': '2017',
    ' FY 2018.2': '2018',
    'FY 2019.2': '2019',
    'FY 2020.2': '2020',
    'FY 2021.2': '2021',
    'FY2022.2': '2022',
    'FY 2023 Estimated.2': '2023'
})

In [5]:
rain_pct_50_df = rain_pct_df.loc["50-state median", :].to_frame().reset_index()
rain_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}, inplace = True
)
rain_pct_50_df["percent"] = 100 * rain_pct_50_df["fraction"]
rain_pct_50_df["series"] = "50-state median Rainy Day Fund"

rain_pct_tn_df = rain_pct_df.loc["Tennessee", :].to_frame().reset_index()
rain_pct_tn_df = rain_pct_tn_df.rename(
    columns = {"index":"year", "Tennessee":"fraction"}
)
rain_pct_tn_df["percent"] = 100 * rain_pct_tn_df["fraction"]
rain_pct_tn_df["series"] = "Tennessee Rainy Day Fund"

totbal_pct_50_df = \
    totbal_pct_df.loc["50-state median", :].to_frame().reset_index()
totbal_pct_50_df = totbal_pct_50_df.rename(
    columns = {"index":"year", "50-state median":"fraction"}
)
totbal_pct_50_df["percent"] = 100 * totbal_pct_50_df["fraction"]
totbal_pct_50_df["series"] = "50-state median total reserve balances"

totbal_pct_tn_df = totbal_pct_df.loc["Tennessee", :].to_frame().reset_index()
totbal_pct_tn_df = totbal_pct_tn_df.rename(
    columns = {'index':'year', "Tennessee":"fraction"}
)
totbal_pct_tn_df["percent"] = 100 * totbal_pct_tn_df["fraction"]
totbal_pct_tn_df["series"] = "Tennessee total reserve balances"

# Merge the four DataFrames to save the joint dataframe as a .csv
fig4_source_df = rain_pct_50_df[['year', 'percent']].rename(
    columns={'percent':'rain_50m_pct'}
)
fig4_source_df = pd.merge(
    fig4_source_df,
    rain_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'rain_tn_pct'}
    ),
    on='year', how='inner'
)
fig4_source_df = pd.merge(
    fig4_source_df,
    totbal_pct_50_df[['year', 'percent']].rename(
        columns={'percent':'totbal_50m_pct'}
    ),
    on='year', how='inner'
)
fig4_source_df = pd.merge(
    fig4_source_df,
    totbal_pct_tn_df[['year', 'percent']].rename(
        columns={'percent':'totbal_tn_pct'}
    ),
    on='year', how='inner'
)
fig4_source_df.to_csv('./data/fig4_source.csv', index=False)

fig4_source_df

,year,rain_50m_pct,rain_tn_pct,totbal_50m_pct,totbal_tn_pct
0,2000,4.127259,2.504361,8.766520,3.285552
1,2001,4.614270,2.537384,7.148315,2.983564
2,2002,1.655668,2.368376,2.887308,2.521389
3,2003,0.722015,2.249264,2.656212,3.059252
4,2004,1.844196,2.654402,5.246650,9.323433
5,2005,2.459819,3.022189,9.016673,8.094287
6,2006,4.553273,3.585613,11.801597,11.793910
7,2007,4.738918,5.553680,11.327766,15.849829
8,2008,4.834502,6.835208,8.292193,10.004921
9,2009,2.728263,5.212919,4.582794,5.936077


In [6]:
# Create Bokeh plot Fig 4: Tennessee Rainy Day fund and total reserves as a
# percentage of general fund expenditures: 2000-2023
fig4_title_lst = [
    ("Figure 4. Tennessee Rainy Day fund and total reserves as a percentage"),
    ("of general fund expenditures: 2000-2023"),
]
# fig4_title_lst = []
fig4_title_short = "Figure 4. TN Rainy Day fund and total reserves"
filename_fig4 = "fig4_tn_raintotbal_tseries.html"
output_file(
    os.path.join(image_dir, filename_fig4),
    title=fig4_title_short,
    mode="inline",
)
output_notebook()

# Format the tooltip
tooltips_fig4 = [
    ("Series:", "@series"),
    ("Year:", "@year"),
    ("Pct. of Gen. Fund Exps.:", "@percent{0.0%}")
]

min_year = 2000
max_year = 2023
min_pct = 2.0
max_pct = 0.0
pct_buffer = 0.05
fig4_height = 500
fig4_width = 800

var_list = [rain_pct_tn_df, rain_pct_50_df, totbal_pct_tn_df, totbal_pct_50_df]
color_list = ["#3477A5", "#D5AB53", "#3477A5", "#D5AB53"]
marker_list = ["circle", "circle", "square", "square"]
legend_label_list = [
    "Rainy day fund, Tennessee",
    "Rainy day fund, 50-state median",
    "Total balances, Tennessee",
    "Total balances, 50-state median"
]
cds_list = []

for k, df in enumerate(var_list):
    min_pct = np.minimum(min_pct, df["percent"].min())
    max_pct = np.maximum(max_pct, df["percent"].max())
    cds_list.append(ColumnDataSource(df))

fig4 = figure(
    height=fig4_height,
    width=fig4_width,
    x_axis_label='Year',
    x_range=(min_year - 1, max_year + 1.6),
    y_axis_label='Percent of general fund expenditures',
    y_range=(min_pct - pct_buffer * (max_pct - min_pct),
             max_pct + pct_buffer * (max_pct - min_pct)),
    tools=["save", "help"],
    toolbar_location="right"
)
fig4.toolbar.logo = None

# Turn off any pan, drag, or scrolling ability
fig4.toolbar.active_drag = None
fig4.toolbar.active_scroll = None
fig4.toolbar.active_tap = None

# Set title font size and axes font sizes
fig4.xaxis.axis_label_text_font_size = '12pt'
fig4.xaxis.major_label_text_font_size = '12pt'
fig4.xaxis.major_label_text_color = '#434244'
fig4.yaxis.axis_label_text_font_size = '12pt'
fig4.yaxis.major_label_text_font_size = '12pt'
fig4.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig4.xaxis.ticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
fig4.xaxis.axis_line_color = '#434244'
fig4.xaxis.major_tick_line_color = '#434244'
fig4.xaxis.minor_tick_line_color = '#434244'
fig4.xgrid.ticker = SingleIntervalTicker(interval=2)
fig4.yaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig4.yaxis.axis_line_color = '#434244'
fig4.yaxis.major_tick_line_color = '#434244'
fig4.yaxis.minor_tick_line_color = '#434244'
fig4.ygrid.ticker = SingleIntervalTicker(interval=10)

# Create lines and markers for time series
for k, yvar in enumerate(var_list):
    fig4.line(x='year', y='percent', source=cds_list[k], color=color_list[k],
              line_width=4, alpha=0.7)
    fig4.scatter(x='year', y='percent', source=cds_list[k], size=10,
                 line_width=1, line_color='black', fill_color=color_list[k],
                 marker=marker_list[k], line_alpha=0.7, fill_alpha=0.7,
                 legend_label=legend_label_list[k])

fig4.segment(x0=2022.4, y0=min_pct - pct_buffer * (max_pct - min_pct),
             x1=2022.4, y1=max_pct + pct_buffer * (max_pct - min_pct),
             color='#434244', line_dash='6 2', line_width=2)

label_temp = Label(
    x=2022.5, y=17.0, x_units='data', y_units='data', text='Projected',
    text_font_size='9pt', text_color='#434244'
)
fig4.add_layout(label_temp)

# Add legend
fig4.legend.location = 'top_center'
fig4.legend.border_line_width = 1
fig4.legend.border_line_color = '#434244'
fig4.legend.border_line_alpha = 1
fig4.legend.label_text_font_size = '12pt'
fig4.legend.label_text_color = '#434244'

# Add title and subtitle to the plot doing reverse loop through items in
# fig_title_lst
for title_line_str in fig4_title_lst[::-1]:
    fig4.add_layout(
        Title(
            text=title_line_str,
            text_font_style="bold",
            text_color='#434244',
            text_font_size="15pt",
            align="center",
        ),
        "above",
    )

# Add notes below image
note_text_list4 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and '+
        'Analysis," Feb. 16, 2024,'
    ),
    ('        accessed July 3, 2024.')
]
for note_text in note_text_list4:
    caption = Title(
        text=note_text, align='left', text_font_size='11pt',
        text_font_style='italic',
        text_color='#434244',
        # text_font='Open Sans'
    )
    fig4.add_layout(caption, 'below')

# Add the HoverTool to the figure
fig4.add_tools(
    HoverTool(
        tooltips=tooltips_fig4,
        visible=False
    )
)

show(fig4)

Loading BokehJS ...

### 1.2. Figure 5. Estimated 2023 Rainy Day Fund Balances and Total Reserves and Balances as Percentages of General-Fund Expenditures

In [7]:
rain_pct_2023_df = rain_pct_df.loc[:'Wyoming', '2023'].to_frame().reset_index()
rain_pct_2023_df = rain_pct_2023_df.rename(
    columns = {"index": "state", "2023": "rain_frac"}
)
rain_pct_2023_df["rain_pct"] = 100 * rain_pct_2023_df["rain_frac"]
rain_pct_2023_df = rain_pct_2023_df.drop('rain_frac', axis=1)

rain_dol_2023_df = rain_dol_df.loc[:'Wyoming', '2023'].to_frame().reset_index()
rain_dol_2023_df = rain_dol_2023_df.rename(
    columns = {"index": "state", "2023": "rain_dol"}
)

totbal_pct_2023_df = \
    totbal_pct_df.loc[:'Wyoming', '2023'].to_frame().reset_index()
totbal_pct_2023_df = totbal_pct_2023_df.rename(
    columns = {"index": "state", "2023": "totbal_frac"}
)
totbal_pct_2023_df["totbal_pct"] = 100 * totbal_pct_2023_df["totbal_frac"]
totbal_pct_2023_df = totbal_pct_2023_df.drop('totbal_frac', axis=1)

totbal_dol_2023_df = \
    totbal_dol_df.loc[:'Wyoming', '2023'].to_frame().reset_index()
totbal_dol_2023_df = totbal_dol_2023_df.rename(
    columns = {"index": "state", "2022": "totbal_dol"}
)

# Merge the four DataFrames
rain_totbal_pct_dol_2023_df = rain_pct_2023_df.copy()
rain_totbal_pct_dol_2023_df = pd.merge(
    rain_totbal_pct_dol_2023_df, rain_dol_2023_df, on='state', how='inner'
)
rain_totbal_pct_dol_2023_df["rain_color"] = "#D5AB53"
rain_totbal_pct_dol_2023_df["rain_color"][
    rain_totbal_pct_dol_2023_df["state"]=="Tennessee"
] = "#EED17E"
rain_totbal_pct_dol_2023_df = pd.merge(
    rain_totbal_pct_dol_2023_df, totbal_pct_2023_df, on='state', how='inner'
)
rain_totbal_pct_dol_2023_df = pd.merge(
    rain_totbal_pct_dol_2023_df, totbal_dol_2023_df, on='state', how='inner'
)
rain_totbal_pct_dol_2023_df["totbal_color"] = "#3477A5"
rain_totbal_pct_dol_2023_df["totbal_color"][
    rain_totbal_pct_dol_2023_df["state"]=="Tennessee"
] = "#74AAC7"

# Create a new variable that is totbal_pct - rain_pct
rain_totbal_pct_dol_2023_df["totbal_rain_pct_dif"] = (
    rain_totbal_pct_dol_2023_df["totbal_pct"] -
    rain_totbal_pct_dol_2023_df["rain_pct"]
)

# Sort by total reserves balances percent
rain_totbal_pct_dol_2023_sorted_df = rain_totbal_pct_dol_2023_df.sort_values(
    by=['totbal_pct', 'rain_pct', 'state'], ascending=[True, True, True]
)

rain_totbal_pct_dol_2023_sorted_df.to_csv(
    os.path.join(data_dir, "fig5_source.csv"), index=False
)
rain_totbal_pct_dol_2023_sorted_df

/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_51481/750146888.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rain_totbal_pct_dol_2023_df["rain_color"][
/var/folders/d4/trj3dssd6s3g8kxvjmczz11w0000gn/T/ipykernel_51481/75014

,state,rain_pct,rain_dol,rain_color,totbal_pct,2023,totbal_color,totbal_rain_pct_dif
12,Illinois,3.780102,1926.000000,#D5AB53,4.133383,2106.000000,#3477A5,3.532806e-01
23,Mississippi,9.202522,579.400000,#D5AB53,9.202522,579.400000,#3477A5,-1.243450e-14
40,South Dakota,9.660563,220.000000,#D5AB53,9.660563,220.000000,#3477A5,0.000000e+00
38,Rhode Island,5.446792,271.045069,#D5AB53,10.478118,521.415610,#3477A5,5.031326e+00
25,Montana,2.345598,118.000000,#D5AB53,12.296499,618.600000,#3477A5,9.950901e+00
46,Washington,2.070403,650.800000,#D5AB53,12.912975,4059.000000,#3477A5,1.084257e+01
5,Colorado,11.136886,1992.400000,#D5AB53,13.753417,2460.500000,#3477A5,2.616531e+00
6,Connecticut,15.317656,3343.400000,#D5AB53,15.317656,3343.400000,#3477A5,0.000000e+00
28,New Hampshire,15.691243,253.900000,#D5AB53,15.691243,253.900000,#3477A5,0.000000e+00
45,Virginia,11.975850,3801.398350,#D5AB53,16.542641,5250.998350,#3477A5,4.566791e+00


In [8]:
# Create Bokeh plot Fig 5: State 2023 Rainy day Fund Balances and Total Fund
# Balances as Percent of General Fund Expenditures
fig5_title_lst = [
    ("Figure 5. State 2023 Rainy day Fund Balances and Total Fund Balances"),
    ("as Percent of General Fund Expenditures")
]
# fig5_title_lst = []
fig5_title_short = "Figure 5. State 2023 Rainy Day fund and Total Reserves"
filename_fig5 = "fig5_rain_totbal_pct_2023.html"
output_file(
    os.path.join(image_dir, filename_fig5),
    title=fig5_title_short,
    mode="inline",
)
output_notebook()

states = rain_totbal_pct_dol_2023_sorted_df['state'].tolist()
legend_label_list_fig5 = ["Rainy day fund", "Total reserves and balances"]
rain_totbal_pct_dol_2023_cds = \
    ColumnDataSource(rain_totbal_pct_dol_2023_sorted_df)

min_pct = rain_totbal_pct_dol_2023_sorted_df['rain_pct'].min()
max_pct = rain_totbal_pct_dol_2023_sorted_df['totbal_pct'].max()
pct_buffer = 0.05
fig5_height = 900
fig5_width = 550

fig5 = figure(
    height=fig5_height,
    width=fig5_width,
    y_range=states,
    x_axis_label='Percentage of general fund expenditures',
    x_range=(min_pct - pct_buffer * (max_pct - min_pct),
             max_pct + pct_buffer * (max_pct - min_pct)),
    tools=["save", "help"],
    toolbar_location="right"
)
fig5.toolbar.logo = None

# Turn off any pan, drag, or scrolling ability
fig5.toolbar.active_drag = None
fig5.toolbar.active_scroll = None
fig5.toolbar.active_tap = None

# Set title font size and axes font sizes
fig5.xaxis.axis_label_text_font_size = '11pt'
fig5.xaxis.major_label_text_font_size = '11pt'
fig5.xaxis.major_label_text_color = '#434244'
fig5.yaxis.axis_label_text_font_size = '9pt'
fig5.yaxis.major_label_text_font_size = '9pt'
fig5.yaxis.major_label_text_color = '#434244'

# Modify tick intervals for X-axis and Y-axis
fig5.xaxis.ticker = SingleIntervalTicker(interval=10, num_minor_ticks=5)
fig5.xaxis.axis_line_color = '#434244'
fig5.xaxis.major_tick_line_color = '#434244'
fig5.xaxis.minor_tick_line_color = '#434244'
fig5.xgrid.ticker = SingleIntervalTicker(interval=10)
fig5.yaxis.axis_line_color = '#434244'
fig5.yaxis.major_tick_line_color = '#434244'
fig5.yaxis.minor_tick_line_color = '#434244'

fig5.hbar_stack(
    ['rain_pct', 'totbal_rain_pct_dif'], y='state', height=0.9,
    color=['rain_color', 'totbal_color'],
    source=rain_totbal_pct_dol_2023_cds, legend_label=legend_label_list_fig5
)

# Add information on hover
tooltips_fig5 = [
    ('State', '@state'),
    ('Rainy day fund % exps.','@rain_pct{0.0}' + '%'),
    ('Total balances % exps.','@totbal_pct{0.0}' + '%'),
]
fig5.add_tools(HoverTool(tooltips=tooltips_fig5, visible=False))

# Add legend
fig5.legend.location = (190, 240)
fig5.legend.border_line_width = 1
fig5.legend.border_line_color = '#434244'
fig5.legend.border_line_alpha = 1
fig5.legend.label_text_color = '#434244'
fig5.legend.label_text_font_size = '11pt'
fig5.legend.label_text_color = '#434244'
fig5.y_range.range_padding = 0.02

# Add title and subtitle to the plot doing reverse loop through items in
# figx_title_lst
for title_line_str in fig5_title_lst[::-1]:
    fig5.add_layout(
        Title(
            text=title_line_str,
            text_font_style="bold",
            text_color='#434244',
            text_font_size="13pt",
            align="center",
        ),
        "above",
    )

# Add notes below image
note_text_list5 = [
    (
        'Source: Pew Charitable Trusts, "Fiscal 50: State Trends and '+
        'Analysis,"'
    ),
    ('        Feb. 16, 2024, accessed July 3, 2024.'),
    (
        'Note: For states in which the blue bar is not visible for total ' +
        'balances'
    ),
    (
        '        and reserves, the value equals the rainy day fund balance'
    ),
    (
        '        percentage.'
    )
]
for note_text in note_text_list5:
    caption = Title(
        text=note_text, align='left', text_font_size='10pt',
        text_font_style='italic', text_color='#434244'
    )
    fig5.add_layout(caption, 'below')

show(fig5)

Loading BokehJS ...

## 2. Tennessee Business Tax Landscape
Important features of TN business tax system:
* 4 main components:
  1. "Excise Tax"
    * Essentailly a CIT
    * Rate is 6.5%
    * Depreciation handled under same rules as IRC and follows federal bonus depreciation bonus depreciation.
  2. Franchise tax
    * Tax on the value of assets of the business
    * Rate is 0.25%
  3. Gross receipts tax
    * TVA
    * mixed drinks
    * vending machines
    * bottlers
  4. Business tax
    * Tax on the gross receipts of the business
    * Some industries (e.g., utilities) handled differently
    * Rate varies (see [here, p. 62](https://www.tn.gov/content/dam/tn/revenue/documents/tax_manuals/august-2022/Business-Tax.pdf))
      * Varies between 0.02% and 0.1% (depends on industry)

### Table 3. Tennessee business taxes: dollars and percent of total tax revenue

## 3. Investment Incentives using Cost of Capital Calculator
Computing the cost-of-capital at the state-level:
  * the statutory rate used will be the sum of the federal and state CIT
    * But can deduct state taxes at federal level
    * So maybe rate is Fed Rate + State rate - fed rate * state rate?
  * franchise tax can use the wealth tax parameter in CCC
  * The interaction of state and federal is not something I want to rush into CCC, so we'll do most of the calculations here by functions defined in this notebook (with a bit of assistance from a couple CCC functions.


Equation for the cost of capital at when considering state taxes in TN:
$$
\rho = \left[\frac{(r-\pi+\delta)(1-k-u^fz^f-u^sz^s + u^fu^sz^s)}{(1-u^f-u^s+u^fu^s)} + w \right] / (1-\tau^{GR}) - \delta
$$
where:
* $\rho$ = cost of capital
* $r$ = nominal discount rate
* $\pi$ = inflation rate
* $k$ = investment tax credit rate
* $u^f$ = statutory CIT rate at the federal level 
* $u^s$ = statutory CIT rate at the state level 
* $z^f$ = NPV of depreciation deduction under federal system
* $z^s$ = NPV of depreciation deduction under state system
* $w$ = property tax rate
* $\tau^{GR}$ = gross receipts tax rate

Calibration:

| Parameter     | Value | Source |
| ----------- | ----------- |----------- |
| $r$     | 0.06      | To give real return of 4\%|
| $\pi$  | 0.02      | Fed target|
| $k$  | 0.0        | No general invest tax credit at federal or state level|
| $u^f$  | 0.21       | Federal IRC|
| $u^s$  | 0.065       | Excise tax rate in TN law|
| $z^f$  | Varies        | Federal IRC|
| $z^s$  |   Varies      | Like Federal IRC, but no bonus deprec|
| $w$  | 0.0025       | Franchise tax rate in TN law |
| $\tau^{GR}$  | 0.0002-0.001       | Bus tax rate in TN law|

### Setup

Create objects to work with

In [9]:
# Creating an instance of the Assets class and call it "assets"
assets = ccc.data.Assets()
# Create an instance of the DepreciationParams
# class object and call it dp
dp = ccc.parameters.DepreciationParams()

In [10]:
# define new cost of capital function to take into account state taxes
def eq_coc_state(delta, z_f, z_s, w, u_f, u_s, tau_GR, inv_tax_credit, pi, r):
    r'''
    Compute the cost of capital
    .. math::
        \rho = \frac{\frac{(r-\pi+\delta)}{1-u^f-u^s+u^fu^s}(1-u^fz^f-u^sz^s+u^fu^sz^s)+w}{(1-\tau^GR)}-\delta
    Args:
        delta (array_like): rate of economic depreciation
        z_f (array_like): net present value of depreciation deductions for
            $1 of investment under the federal system
        z_s (array_like): net present value of depreciation deductions for
            $1 of investment under the state system
        w (scalar): property tax rate
        u_f (scalar): statutory marginal tax rate for the first layer of
            income taxes at the federal level
        u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
        tau_GR (scalar): gross receipts tax
        inv_tax_credit (scalar): investment tax credit rate
        pi (scalar): inflation rate
        r (scalar): discount rate
    Returns:
        rho (array_like): the cost of capital
    '''
    rho = (
        ((((r - pi + delta) / (1 - u_f - u_s + u_f * u_s)) *
          (1 - inv_tax_credit - u_f * z_f - u_s * z_s + u_f * u_s * z_s) + w) /
         (1 - tau_GR)) - delta
    )

    return rho

In [11]:
# Define model parameters
inflation_rate = 0.02 # Inflation rate
nominal_int_rate = 0.06
fraction_financed_w_debt = 0.0
federal_bonus_depreciation = {"machines": 0.4, "buildings": 0.0, "intangibles": 0.4}
depreciation_rates = {"machines": 0.1031, "buildings": 0.0314, "intangibles": 0.33}
# Example machine: EI40, example building: SI00, example intangible: ENS3
depreciation_lives = {"machines": 7, "buildings": 39, "intangibles": 3}
depreciation_methods = {"machines": "dbsl", "buildings": "sl", "intangibles": "sl"}
E = 0.06  # Expected after-tax return on corporate equity
profit_rate = 0.2 # 0.2 is a 20% profit rate
int_haircut = 0.0
u_f = 0.21
u_s = 0.065
franchise_tax_rate = 0.0025
tau_GR = 0.001 # this is the higher end 0.0002-0.001 is what TF reports, varies by industry
inv_tax_credit = 0.0
bonus_s = {"machines": 1.0, "buildings": 0.0, "intangibles": 1.0}


In [12]:
# compute outputs
def compute_outputs(u_s, bonus_s, franchise_tax_rate, tau_GR):
    """
    This function computes the outputs of interest and allows one to change the
    parameters of TN state law

    Args:
      u_s (scalar): statutory marginal tax rate for the first layer of
            income taxes at the state level
      bonus_s (dict): rates of bonus depreciation in TN by asset type
      franchise_tax_rate (scalar): TN franchise tax rate
      excise_tax_rate (scalar): TN excise tax rate
      tau_GR (scalar): gross receipts tax
    """
    out_dict = {"machines": {}, "buildings": {}, "intangibles": {}}
    for k, v in depreciation_rates.items():
        r = ccc.paramfunctions.calc_r(
            u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
            nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
        r_prime = ccc.paramfunctions.calc_r_prime(
            nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
        if depreciation_methods[k] == "dbsl":
            z_f = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.dbsl(depreciation_lives[k], 2, bonus_s[k], r)
        elif depreciation_methods[k] == "sl":
            z_f = ccc.calcfunctions.sl(depreciation_lives[k], federal_bonus_depreciation[k], r)
            z_s = ccc.calcfunctions.sl(depreciation_lives[k], bonus_s[k], r)
        else:
          print("Please enter one of: dbsl, sl")
          assert False
        rho = eq_coc_state(
            depreciation_rates[k], z_f, z_s, franchise_tax_rate, u_f, u_s,
            tau_GR, inv_tax_credit, inflation_rate, r
        )
        metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
        eatr = ccc.calcfunctions.eq_eatr(
            rho, metr, profit_rate, u_f + u_f - u_f * u_s
        )
        out_dict[k]["rho"] = rho
        out_dict[k]["metr"] = metr
        out_dict[k]["eatr"] = eatr
    return out_dict

In [13]:
# Current law
base_df = pd.DataFrame(compute_outputs(
    u_s, federal_bonus_depreciation, franchise_tax_rate, tau_GR
))
# Set TN bonus depreciation to 100 percent
tnbonus_df = pd.DataFrame(compute_outputs(
    u_s, bonus_s, franchise_tax_rate, tau_GR
))
# Reduce the TN excise tax rate from 6.5 percent to 6.0 percent
excise_df = pd.DataFrame(compute_outputs(
    0.060, federal_bonus_depreciation, franchise_tax_rate, tau_GR
))
franchise_df = pd.DataFrame(compute_outputs(
    u_s, federal_bonus_depreciation, 0.0020, tau_GR
))
# Repeal TN business tax and gross receipts tax
repealBusGR_df = pd.DataFrame(compute_outputs(
    u_s, federal_bonus_depreciation, franchise_tax_rate, 0.0
))

In [14]:
# Create policy names and put in one dataframe
base_df['Policy'] = "Current Law"
repealBusGR_df['Policy'] = "Repeal Business and Gross Receipts Tax"
tnbonus_df['Policy'] = "Set TN bonus depreciation to 100 percent"
excise_df['Policy'] = "Reduce excise tax rate from 0.065 to 0.060"
franchise_df['Policy'] = "Reduce franchise tax rate from 0.0025 to 0.0020"
# append dataframes together
df = pd.concat([base_df, repealBusGR_df, tnbonus_df, excise_df, franchise_df])
df.reset_index(inplace=True)
df.rename(columns={"index": "output_var"}, inplace=True)
df = pd.melt(df, id_vars=["Policy", "output_var"], var_name="asset_type")

In [15]:
# Plot Table 4: Effects of Tennessee Business Tax Reforms in 2025 on Business
# Incentives to Invest and on State Tax Revenue
fig6 = px.bar(
    df[(df["output_var"]=="metr") & (df["asset_type"]=="machines")],
    x="Policy", y="value", color='Policy',
    labels={'value':'Marginal Effective Tax Rate'}, height=400
)
fig6.show()

In [16]:
# Other results
fig7 = px.bar(
    df[(df["output_var"]=="rho") & (df["asset_type"]=="machines")], x="Policy",
    y="value", color='Policy', labels={'value':'Cost of Capital'}, height=400
)
fig7.show()

In [17]:
# Other results
fig9 = px.histogram(
    df[
        (df["output_var"]=="metr") &
        (df['Policy'].isin([
            "Current Law",
            "Repeal Business Tax", "Follow Federal Bonus Depreciation",
            "Repeal Franchise Tax"
        ]))
    ], x="asset_type", y="value", color='Policy', barmode='group',
    labels={'asset_type':'Asset Type', 'value': 'Marginal Effective Tax Rate'},
    height=400)
fig9.show()

In [18]:
# compute example results for equipment
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 0.8, r)  # 5 year asset, double declining balance, 80% bonus
z_s = ccc.calcfunctions.dbsl(5, 2, 0.0, r)  # 5 year asset, double declining balance, no bonus
rho = eq_coc_state(
        depreciation_rates['machines'], z_f, z_s, franchise_tax_rate,
        u_f, u_s, tau_GR, inv_tax_credit, inflation_rate, r)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

Machines: rho =  0.06565086927715877 , metr =  0.08607455376260828  and eatr =  0.30121819273192646


In [19]:
# compare to federal
r = ccc.paramfunctions.calc_r(
        u_f + u_s - u_f * u_s, nominal_int_rate, inflation_rate,
        nominal_int_rate, fraction_financed_w_debt, int_haircut, E, 0.0)
r_prime = ccc.paramfunctions.calc_r_prime(
    nominal_int_rate, inflation_rate, fraction_financed_w_debt, E)
z_f = ccc.calcfunctions.dbsl(5, 2, 0.8, r)  # 5 year asset, double declining balance, 80% bonus
psi_f = 0.5
nu_f = 0.05
rho = ccc.calcfunctions.eq_coc(
    depreciation_rates['machines'], z_f, 0.0, u_f, 0.0, inv_tax_credit, psi_f,
    nu_f, inflation_rate, r
)
metr = ccc.calcfunctions.eq_metr(rho, r_prime, inflation_rate)
eatr = ccc.calcfunctions.eq_eatr(rho, metr, profit_rate, u_f + u_f - u_f * u_s)
print('Machines: rho = ', rho, ', metr = ', metr, ' and eatr = ', eatr)

Machines: rho =  0.10335569620253163 , metr =  0.41948047176397096  and eatr =  0.41313554525316454
